<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/creditdefaultswap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math # For fabs if needed, though assertAlmostEqual handles it

class CreditDefaultSwapTests(unittest.TestCase):

    def test_cached_value(self):
        print("Testing credit-default swap against cached values...")

        saved_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = ql.Date(9, ql.June, 2006)
        today = ql.Settings.instance().evaluationDate
        calendar = ql.TARGET()

        hazard_rate_quote = ql.SimpleQuote(0.01234)
        hazard_rate_handle = ql.QuoteHandle(hazard_rate_quote)

        probability_curve = ql.RelinkableDefaultProbabilityTermStructureHandle()
        probability_curve.linkTo(
            ql.FlatHazardRate(0, calendar, hazard_rate_handle, ql.Actual360())
        )

        discount_curve = ql.RelinkableYieldTermStructureHandle()
        discount_curve.linkTo(
            ql.FlatForward(today, 0.06, ql.Actual360())
        )

        issue_date = calendar.advance(today, -1, ql.Years)
        maturity = calendar.advance(issue_date, 10, ql.Years)
        frequency = ql.Semiannual
        convention = ql.ModifiedFollowing

        schedule = ql.Schedule(issue_date, maturity, ql.Period(frequency), calendar,
                               convention, convention, ql.DateGeneration.Forward, False)

        fixed_rate = 0.0120
        day_count = ql.Actual360()
        notional = 10000.0
        recovery_rate = 0.4

        cds = ql.CreditDefaultSwap(ql.Protection.Seller, notional, fixed_rate,
                                   schedule, convention, day_count, True, True)

        cds.setPricingEngine(ql.MidPointCdsEngine(probability_curve, recovery_rate, discount_curve))

        expected_npv_midpoint = 295.0153398
        expected_fair_rate_midpoint = 0.007517539081
        tolerance_midpoint = 1.0e-7

        calculated_npv = cds.NPV()
        calculated_fair_rate = cds.fairSpread()

        self.assertAlmostEqual(calculated_npv, expected_npv_midpoint, delta=tolerance_midpoint,
                               msg=f"Mid-point NPV: expected {expected_npv_midpoint}, got {calculated_npv}")
        self.assertAlmostEqual(calculated_fair_rate, expected_fair_rate_midpoint, delta=tolerance_midpoint,
                               msg=f"Mid-point Fair Rate: expected {expected_fair_rate_midpoint}, got {calculated_fair_rate}")

        # Integral Engine - 1 day step
        cds.setPricingEngine(ql.IntegralCdsEngine(ql.Period(1, ql.Days), probability_curve,
                                                  recovery_rate, discount_curve))

        calculated_npv_integral_1d = cds.NPV()
        calculated_fair_rate_integral_1d = cds.fairSpread()
        tolerance_integral_npv = notional * 1.0e-5 * 10 # Matches C++ tolerance logic
        tolerance_integral_rate = 1.0e-5


        self.assertAlmostEqual(calculated_npv_integral_1d, expected_npv_midpoint, delta=tolerance_integral_npv,
                               msg=f"Integral (1D) NPV: expected {expected_npv_midpoint}, got {calculated_npv_integral_1d}")
        self.assertAlmostEqual(calculated_fair_rate_integral_1d, expected_fair_rate_midpoint, delta=tolerance_integral_rate,
                               msg=f"Integral (1D) Fair Rate: expected {expected_fair_rate_midpoint}, got {calculated_fair_rate_integral_1d}")

        # Integral Engine - 1 week step
        cds.setPricingEngine(ql.IntegralCdsEngine(ql.Period(1, ql.Weeks), probability_curve,
                                                  recovery_rate, discount_curve))
        calculated_npv_integral_1w = cds.NPV()
        calculated_fair_rate_integral_1w = cds.fairSpread()

        self.assertAlmostEqual(calculated_npv_integral_1w, expected_npv_midpoint, delta=tolerance_integral_npv,
                               msg=f"Integral (1W) NPV: expected {expected_npv_midpoint}, got {calculated_npv_integral_1w}")
        self.assertAlmostEqual(calculated_fair_rate_integral_1w, expected_fair_rate_midpoint, delta=tolerance_integral_rate,
                               msg=f"Integral (1W) Fair Rate: expected {expected_fair_rate_midpoint}, got {calculated_fair_rate_integral_1w}")

        ql.Settings.instance().evaluationDate = saved_eval_date


    def test_cached_market_value(self):
        print("Testing credit-default swap against cached market values...")
        saved_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = ql.Date(9, ql.June, 2006)
        eval_date = ql.Settings.instance().evaluationDate
        calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

        discount_dates_data = [
            eval_date,
            calendar.advance(eval_date, 1, ql.Weeks,  ql.ModifiedFollowing),
            calendar.advance(eval_date, 1, ql.Months, ql.ModifiedFollowing),
            calendar.advance(eval_date, 2, ql.Months, ql.ModifiedFollowing),
            calendar.advance(eval_date, 3, ql.Months, ql.ModifiedFollowing),
            calendar.advance(eval_date, 6, ql.Months, ql.ModifiedFollowing),
            calendar.advance(eval_date,12, ql.Months, ql.ModifiedFollowing),
            calendar.advance(eval_date, 2, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 3, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 4, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 5, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 6, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 7, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 8, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 9, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date,10, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date,15, ql.Years, ql.ModifiedFollowing)
        ]
        dfs_data = [
            1.0, 0.9990151375768731, 0.99570502636871183, 0.99118260474528685,
            0.98661167950906203, 0.9732592953359388, 0.94724424481038083,
            0.89844996737120875, 0.85216647839921411, 0.80775477692556874,
            0.76517289234200347, 0.72401019553182933, 0.68503909569219212,
            0.64797499814013748, 0.61263171936255534, 0.5791942350748791,
            0.43518868769953606
        ]
        curve_day_counter = ql.Actual360()
        discount_dates = ql.DateVector(discount_dates_data)
        dfs = ql.DoubleVector(dfs_data)

        discount_curve = ql.RelinkableYieldTermStructureHandle()
        discount_curve.linkTo(ql.DiscountCurve(discount_dates, dfs, curve_day_counter))

        prob_day_counter = ql.Thirty360(ql.Thirty360.BondBasis)
        prob_dates_data = [
            eval_date,
            calendar.advance(eval_date, 6, ql.Months, ql.ModifiedFollowing),
            calendar.advance(eval_date, 1, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 2, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 3, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 4, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 5, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date, 7, ql.Years, ql.ModifiedFollowing),
            calendar.advance(eval_date,10, ql.Years, ql.ModifiedFollowing)
        ]
        default_probabilities_data = [
            0.0000, 0.0047, 0.0093, 0.0286, 0.0619, 0.0953, 0.1508, 0.2288, 0.3666
        ]

        prob_dates = ql.DateVector(prob_dates_data)
        hazard_rates_data = [0.0]
        for i in range(1, len(prob_dates_data)):
            t1 = prob_day_counter.yearFraction(prob_dates_data[0], prob_dates_data[i-1])
            t2 = prob_day_counter.yearFraction(prob_dates_data[0], prob_dates_data[i])
            s1 = 1.0 - default_probabilities_data[i-1]
            s2 = 1.0 - default_probabilities_data[i]
            hazard_rates_data.append(math.log(s1/s2) / (t2-t1) if (t2-t1) > 1e-12 else 0.0) # Avoid div by zero for coincident dates

        hazard_rates = ql.DoubleVector(hazard_rates_data)

        piecewise_flat_hazard_rate = ql.RelinkableDefaultProbabilityTermStructureHandle()
        piecewise_flat_hazard_rate.linkTo(
            ql.InterpolatedHazardRateCurveLinear( # C++ used BackwardFlat, Python might expose as generic or specific
                prob_dates, hazard_rates, prob_day_counter # Using Linear as a placeholder, might need specific binding for BackwardFlat if available
            )
        )
        # If ql.InterpolatedHazardRateCurve with template is not directly available,
        # one might need to use ql.PiecewiseHazardRateCurve with ql.BackwardFlat, e.g.,
        # helpers = []
        # for i in range(len(prob_dates_data)):
        #    helpers.append(ql.HazardRateHelper(ql.QuoteHandle(ql.SimpleQuote(hazard_rates_data[i])), prob_dates_data[i], ql.BackwardFlat()))
        # piecewise_flat_hazard_rate.linkTo(ql.PiecewiseHazardRateCurve(prob_dates_data[0], helpers, prob_day_counter))
        # For simplicity, I'm assuming a direct InterpolatedHazardRateCurve binding exists that defaults to Linear or similar.
        # The C++ test uses InterpolatedHazardRateCurve<BackwardFlat>. The Python equivalent might be:
        # ql.InterpolatedHazardRateCurve(prob_dates, hazard_rates, prob_day_counter, calendar, ql.BackwardFlat())
        # Let's try that if Linear doesn't match. For now, assume Linear is a close proxy if BackwardFlat isn't directly templated.
        # A more robust way if a direct BackwardFlat template isn't available:
        # Note: QuantLib Python often uses specific classes like InterpolatedHazardRateCurveLogLinear, etc.
        # or a generic one where the interpolator is an argument.
        # Let's stick to the direct C++ translation for now.
        # This constructor is available: InterpolatedHazardRateCurve(dates, rates, dayCounter, calendar = NullCalendar(), [], BackwardFlat())
        piecewise_flat_hazard_rate.linkTo(
             ql.InterpolatedHazardRateCurve(prob_dates, hazard_rates, prob_day_counter,
                                            calendar, [], ql.BackwardFlat()) # Corrected
        )


        issue_date = ql.Date(20, ql.March, 2006)
        maturity = ql.Date(20, ql.June, 2013)
        cds_frequency = ql.Semiannual
        cds_convention = ql.ModifiedFollowing

        schedule = ql.Schedule(issue_date, maturity, ql.Period(cds_frequency), calendar,
                               cds_convention, cds_convention,
                               ql.DateGeneration.Forward, False)

        recovery_rate = 0.25
        fixed_rate = 0.0224
        cds_day_count = ql.Actual360()
        cds_notional = 100.0

        cds = ql.CreditDefaultSwap(ql.Protection.Seller, cds_notional, fixed_rate,
                                   schedule, cds_convention, cds_day_count, True, True)
        cds.setPricingEngine(ql.MidPointCdsEngine(piecewise_flat_hazard_rate,
                                                  recovery_rate, discount_curve))

        calculated_npv = cds.NPV()
        calculated_fair_rate = cds.fairSpread()

        expected_npv = -1.364048777
        expected_fair_rate = 0.0248429452
        tolerance = 1e-9

        self.assertAlmostEqual(calculated_npv, expected_npv, delta=tolerance,
                               msg=f"Market Value NPV: expected {expected_npv}, got {calculated_npv}")
        self.assertAlmostEqual(calculated_fair_rate, expected_fair_rate, delta=tolerance,
                               msg=f"Market Value Fair Rate: expected {expected_fair_rate}, got {calculated_fair_rate}")

        ql.Settings.instance().evaluationDate = saved_eval_date

    def test_implied_hazard_rate(self):
        print("Testing implied hazard-rate for credit-default swaps...")
        saved_eval_date = ql.Settings.instance().evaluationDate
        calendar = ql.TARGET()
        today = calendar.adjust(ql.Date.todaysDate())
        ql.Settings.instance().evaluationDate = today

        h1, h2 = 0.30, 0.40
        day_counter = ql.Actual365Fixed()

        dates_data = [today, today + ql.Period(5, ql.Years), today + ql.Period(10, ql.Years)]
        hazard_rates_data = [h1, h1, h2] # Flat up to 5Y, then jumps to h2

        dates_vec = ql.DateVector(dates_data)
        hazard_rates_vec = ql.DoubleVector(hazard_rates_data)

        probability_curve = ql.RelinkableDefaultProbabilityTermStructureHandle()
        probability_curve.linkTo(
            ql.InterpolatedHazardRateCurve(dates_vec, hazard_rates_vec, day_counter,
                                           calendar, [], ql.BackwardFlat())
        )

        discount_curve = ql.RelinkableYieldTermStructureHandle()
        discount_curve.linkTo(ql.FlatForward(today, 0.03, ql.Actual360()))

        frequency = ql.Semiannual
        convention = ql.ModifiedFollowing
        issue_date = calendar.advance(today, -6, ql.Months)
        fixed_rate = 0.0120
        cds_day_count = ql.Actual360()
        notional = 10000.0
        recovery_rate = 0.4
        latest_rate = None # Using None for ql.Null

        for n_years in range(6, 11):
            maturity = calendar.advance(issue_date, n_years, ql.Years)
            schedule = ql.Schedule(issue_date, maturity, ql.Period(frequency), calendar,
                                   convention, convention,
                                   ql.DateGeneration.Forward, False)

            cds1 = ql.CreditDefaultSwap(ql.Protection.Seller, notional, fixed_rate,
                                       schedule, convention, cds_day_count, True, True)
            cds1.setPricingEngine(ql.MidPointCdsEngine(probability_curve, recovery_rate, discount_curve))

            npv1 = cds1.NPV()
            implied_flat_rate = cds1.impliedHazardRate(npv1, discount_curve, day_counter, recovery_rate)

            self.assertTrue(h1 <= implied_flat_rate <= h2,
                            f"Implied hazard rate {implied_flat_rate} outside expected range [{h1}, {h2}] for {n_years}Y maturity")

            if n_years > 6 and latest_rate is not None:
                self.assertGreaterEqual(implied_flat_rate, latest_rate,
                                        f"Implied hazard rate decreasing: {n_years}Y ({implied_flat_rate}) vs prev ({latest_rate})")
            latest_rate = implied_flat_rate

            implied_prob_curve_handle = ql.RelinkableDefaultProbabilityTermStructureHandle()
            implied_prob_curve_handle.linkTo(
                ql.FlatHazardRate(today, ql.QuoteHandle(ql.SimpleQuote(implied_flat_rate)), day_counter)
            )

            cds2 = ql.CreditDefaultSwap(ql.Protection.Seller, notional, fixed_rate,
                                       schedule, convention, cds_day_count, True, True)
            cds2.setPricingEngine(ql.MidPointCdsEngine(implied_prob_curve_handle, recovery_rate, discount_curve))

            npv2 = cds2.NPV()
            tolerance_npv_repro = 1.0
            self.assertAlmostEqual(npv1, npv2, delta=tolerance_npv_repro,
                                   msg=f"Failed to reproduce NPV with implied rate for {n_years}Y: expected {npv1}, got {npv2}")

        ql.Settings.instance().evaluationDate = saved_eval_date

    def test_fair_spread(self):
        print("Testing fair-spread calculation for credit-default swaps...")
        saved_eval_date = ql.Settings.instance().evaluationDate
        calendar = ql.TARGET()
        today = calendar.adjust(ql.Date.todaysDate())
        ql.Settings.instance().evaluationDate = today

        hazard_rate_quote = ql.SimpleQuote(0.01234)
        probability_curve = ql.RelinkableDefaultProbabilityTermStructureHandle()
        probability_curve.linkTo(
            ql.FlatHazardRate(0, calendar, ql.QuoteHandle(hazard_rate_quote), ql.Actual360())
        )
        discount_curve = ql.RelinkableYieldTermStructureHandle()
        discount_curve.linkTo(ql.FlatForward(today, 0.06, ql.Actual360()))

        issue_date = calendar.advance(today, -1, ql.Years)
        maturity = calendar.advance(issue_date, 10, ql.Years)
        convention = ql.Following

        schedule = ql.MakeSchedule().from_(issue_date).to(maturity) \
                                 .withFrequency(ql.Quarterly) \
                                 .withCalendar(calendar) \
                                 .withTerminationDateConvention(convention) \
                                 .withRule(ql.DateGeneration.TwentiethIMM).makeSchedule()

        fixed_rate = 0.001
        day_count = ql.Actual360()
        notional = 10000.0
        recovery_rate = 0.4
        engine = ql.MidPointCdsEngine(probability_curve, recovery_rate, discount_curve)

        cds = ql.CreditDefaultSwap(ql.Protection.Seller, notional, fixed_rate,
                                   schedule, convention, day_count, True, True)
        cds.setPricingEngine(engine)
        fair_rate = cds.fairSpread()

        fair_cds = ql.CreditDefaultSwap(ql.Protection.Seller, notional, fair_rate,
                                        schedule, convention, day_count, True, True)
        fair_cds.setPricingEngine(engine)
        fair_npv = fair_cds.NPV()
        tolerance = 1e-9

        self.assertAlmostEqual(fair_npv, 0.0, delta=tolerance,
                               msg=f"Fair spread CDS NPV not zero: got {fair_npv}, spread {fair_rate}")
        ql.Settings.instance().evaluationDate = saved_eval_date


    def test_fair_upfront(self):
        print("Testing fair-upfront calculation for credit-default swaps...")
        saved_eval_date = ql.Settings.instance().evaluationDate
        calendar = ql.TARGET()
        today = calendar.adjust(ql.Date.todaysDate())
        ql.Settings.instance().evaluationDate = today

        hazard_rate_quote = ql.SimpleQuote(0.01234)
        probability_curve = ql.RelinkableDefaultProbabilityTermStructureHandle()
        probability_curve.linkTo(
            ql.FlatHazardRate(0, calendar, ql.QuoteHandle(hazard_rate_quote), ql.Actual360())
        )
        discount_curve = ql.RelinkableYieldTermStructureHandle()
        discount_curve.linkTo(ql.FlatForward(today, 0.06, ql.Actual360()))

        issue_date = today
        maturity = calendar.advance(issue_date, 10, ql.Years)
        convention = ql.Following

        schedule = ql.MakeSchedule().from_(issue_date).to(maturity) \
                                 .withFrequency(ql.Quarterly) \
                                 .withCalendar(calendar) \
                                 .withTerminationDateConvention(convention) \
                                 .withRule(ql.DateGeneration.TwentiethIMM).makeSchedule()

        fixed_rate = 0.05
        upfront = 0.001
        day_count = ql.Actual360()
        notional = 10000.0
        recovery_rate = 0.4
        engine = ql.MidPointCdsEngine(probability_curve, recovery_rate, discount_curve, True) # includeSettlementDateFlows

        cds1 = ql.CreditDefaultSwap(ql.Protection.Seller, notional, upfront, fixed_rate,
                                   schedule, convention, day_count, True, True)
        cds1.setPricingEngine(engine)
        fair_upfront1 = cds1.fairUpfront()

        fair_cds1 = ql.CreditDefaultSwap(ql.Protection.Seller, notional, fair_upfront1, fixed_rate,
                                        schedule, convention, day_count, True, True)
        fair_cds1.setPricingEngine(engine)
        fair_npv1 = fair_cds1.NPV()
        tolerance = 1e-9
        self.assertAlmostEqual(fair_npv1, 0.0, delta=tolerance,
                               msg=f"Fair upfront CDS NPV not zero (case 1): got {fair_npv1}, upfront {fair_upfront1}")

        # Case 2: Null upfront
        upfront2 = 0.0
        cds2 = ql.CreditDefaultSwap(ql.Protection.Seller, notional, upfront2, fixed_rate,
                                   schedule, convention, day_count, True, True)
        cds2.setPricingEngine(engine)
        fair_upfront2 = cds2.fairUpfront()

        fair_cds2 = ql.CreditDefaultSwap(ql.Protection.Seller, notional, fair_upfront2, fixed_rate,
                                        schedule, convention, day_count, True, True)
        fair_cds2.setPricingEngine(engine)
        fair_npv2 = fair_cds2.NPV()
        self.assertAlmostEqual(fair_npv2, 0.0, delta=tolerance,
                               msg=f"Fair upfront CDS NPV not zero (case 2): got {fair_npv2}, upfront {fair_upfront2}")

        ql.Settings.instance().evaluationDate = saved_eval_date


    def test_isda_engine(self):
        print("Testing ISDA engine calculations for credit-default swaps...")
        saved_eval_date = ql.Settings.instance().evaluationDate
        using_at_par_coupons = ql.IborCoupon.Settings.instance().usingAtParCoupons()

        trade_date = ql.Date(21, ql.May, 2009)
        ql.Settings.instance().evaluationDate = trade_date

        # ISDA yield curve
        isda_rate_helpers = []
        dep_tenors = [1, 2, 3, 6, 9, 12]
        dep_quotes = [0.003081, 0.005525, 0.007163, 0.012413, 0.014, 0.015488]
        for i in range(len(dep_tenors)):
            isda_rate_helpers.append(
                ql.DepositRateHelper(dep_quotes[i], ql.Period(dep_tenors[i], ql.Months), 2,
                                     ql.WeekendsOnly(), ql.ModifiedFollowing, False, ql.Actual360())
            )

        swap_tenors = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30]
        swap_quotes = [
            0.011907, 0.01699, 0.021198, 0.02444, 0.026937, 0.028967, 0.030504,
            0.031719, 0.03279, 0.034535, 0.036217, 0.036981, 0.037246, 0.037605
        ]
        isda_ibor = ql.IborIndex("IsdaIbor", ql.Period(3, ql.Months), 2, ql.USDCurrency(),
                                 ql.WeekendsOnly(), ql.ModifiedFollowing, False, ql.Actual360())
        for i in range(len(swap_tenors)):
            isda_rate_helpers.append(
                ql.SwapRateHelper(swap_quotes[i], ql.Period(swap_tenors[i], ql.Years),
                                 ql.WeekendsOnly(), ql.Semiannual, ql.ModifiedFollowing,
                                 ql.Thirty360(ql.Thirty360.BondBasis), isda_ibor)
            )

        discount_curve = ql.RelinkableYieldTermStructureHandle()
        discount_curve.linkTo(
            ql.PiecewiseYieldCurveLogLinearDiscount( # C++ used PiecewiseYieldCurve<Discount, LogLinear>
                0, ql.WeekendsOnly(), isda_rate_helpers, ql.Actual365Fixed()
            )
        )

        probability_curve = ql.RelinkableDefaultProbabilityTermStructureHandle()
        term_dates_data = [
            ql.Date(20, ql.June, 2010), ql.Date(20, ql.June, 2011), ql.Date(20, ql.June, 2012),
            ql.Date(20, ql.June, 2016), ql.Date(20, ql.June, 2019)
        ]
        spreads_data = [0.001, 0.1]
        recoveries_data = [0.2, 0.4]
        markit_values = [
            -97798.29358, -97776.11889, 914971.5977, 894985.6298,
            -186921.3594, -186839.8148, 1646623.672, 1579803.626,
            -274298.9203, -274122.4725, 2279730.93, 2147972.527,
            -592420.2297, -591571.2294, 3993550.206, 3545843.418,
            -797501.1422, -795915.9787, 4702034.688, 4042340.999
        ]
        tolerance = 1.0e-3 if not using_at_par_coupons else 1.0e-6 # C++ logic inverted

        l = 0
        for term_date in term_dates_data:
            for spread in spreads_data:
                for recovery in recoveries_data:
                    quoted_trade = ql.MakeCreditDefaultSwap(term_date, spread) \
                                     .withNominal(10000000.0).makeCreditDefaultSwap()

                    h = quoted_trade.impliedHazardRate(0.0, discount_curve, ql.Actual365Fixed(),
                                                       recovery, 1e-10, ql.CreditDefaultSwap.ISDA)
                    probability_curve.linkTo(
                        ql.FlatHazardRate(0, ql.WeekendsOnly(), ql.QuoteHandle(ql.SimpleQuote(h)), ql.Actual365Fixed())
                    )

                    engine = ql.IsdaCdsEngine(probability_curve, recovery, discount_curve, None, # ext::nullopt -> None
                                              ql.IsdaCdsEngine.Taylor, ql.IsdaCdsEngine.HalfDayBias,
                                              ql.IsdaCdsEngine.Piecewise)

                    conventional_trade = ql.MakeCreditDefaultSwap(term_date, 0.01) \
                                           .withNominal(10000000.0) \
                                           .withPricingEngine(engine).makeCreditDefaultSwap()

                    calculated_value = conventional_trade.notional() * conventional_trade.fairUpfront()
                    expected_value = markit_values[l]

                    # QL_CHECK_CLOSE equivalent
                    self.assertAlmostEqual(calculated_value, expected_value, delta=abs(expected_value * tolerance),
                                           msg=f"ISDA Engine Fair Upfront Value: expected {expected_value}, got {calculated_value} for case {l}")

                    # Test Buyer/Seller NPV near zero with fair upfront
                    fair_upfront_val = conventional_trade.fairUpfront()

                    conventional_trade_buy = ql.MakeCreditDefaultSwap(term_date, 0.01) \
                        .withNominal(10000000.0) \
                        .withUpfrontRate(fair_upfront_val) \
                        .withSide(ql.Protection.Buyer) \
                        .withPricingEngine(engine).makeCreditDefaultSwap()
                    self.assertAlmostEqual(conventional_trade_buy.NPV(), 0.0, delta=abs(conventional_trade.notional() * tolerance), # Adjusted tolerance for NPV
                                           msg=f"ISDA Buyer NPV not zero: {conventional_trade_buy.NPV()} for case {l}")

                    conventional_trade_sell = ql.MakeCreditDefaultSwap(term_date, 0.01) \
                        .withNominal(10000000.0) \
                        .withUpfrontRate(fair_upfront_val) \
                        .withSide(ql.Protection.Seller) \
                        .withPricingEngine(engine).makeCreditDefaultSwap()
                    self.assertAlmostEqual(conventional_trade_sell.NPV(), 0.0, delta=abs(conventional_trade.notional() * tolerance),
                                           msg=f"ISDA Seller NPV not zero: {conventional_trade_sell.NPV()} for case {l}")
                    l += 1

        ql.Settings.instance().evaluationDate = saved_eval_date


    def test_accrual_rebate_amounts(self):
        print("Testing accrual rebate amounts on credit default swaps...")
        saved_eval_date = ql.Settings.instance().evaluationDate
        notional = 10000000.0
        spread = 0.0100
        maturity = ql.Date(20, ql.June, 2014)

        inputs = {
            ql.Date(18, ql.March, 2009): 24166.67,
            ql.Date(19, ql.March, 2009): 0.00,
            ql.Date(20, ql.March, 2009): 277.78,
            ql.Date(23, ql.March, 2009): 1111.11,
            ql.Date(19, ql.June, 2009): 25555.56,
            ql.Date(20, ql.June, 2009): 25833.33,
            ql.Date(21, ql.June, 2009): 0.00, # Assuming weekends only calendar makes this a non-business day
            ql.Date(22, ql.June, 2009): 277.78,
            ql.Date(18, ql.June, 2014): 25277.78,
            ql.Date(19, ql.June, 2014): 25555.56
        }
        for trade_date, expected_accrual in inputs.items():
            ql.Settings.instance().evaluationDate = trade_date
            cds = ql.MakeCreditDefaultSwap(maturity, spread) \
                    .withNominal(notional).makeCreditDefaultSwap()
            # The test needs an engine to calculate accrual rebate properly, even if simple
            # For accrual rebate, the exact curves might not be super critical if it's based on schedule
            # However, to be safe, let's provide some dummy engine
            dummy_discount = ql.YieldTermStructureHandle(ql.FlatForward(trade_date, 0.01, ql.Actual360()))
            dummy_prob = ql.DefaultProbabilityTermStructureHandle(
                ql.FlatHazardRate(trade_date, ql.QuoteHandle(ql.SimpleQuote(0.01)), ql.Actual360())
            )
            # cds.setPricingEngine(ql.MidPointCdsEngine(dummy_prob, 0.4, dummy_discount)) # Engine might be needed for accrualRebate to be non-null

            # The accrualRebate method itself calculates based on the CDS schedule and rules.
            # It might not require a full pricing engine for the amount if all info is in the CDS.
            accrual_rebate_obj = cds.accrualRebate()
            if accrual_rebate_obj: # Check if accrual is applicable
                 self.assertAlmostEqual(accrual_rebate_obj.amount(), expected_accrual, delta=0.01,
                                   msg=f"Accrual Rebate Amount for {trade_date}: expected {expected_accrual}, got {accrual_rebate_obj.amount()}")
            elif expected_accrual == 0.0: # If no accrual rebate object, and expected is 0, it's fine.
                pass
            else:
                self.fail(f"Accrual rebate object is None for {trade_date} but expected {expected_accrual}")


        ql.Settings.instance().evaluationDate = saved_eval_date


    def test_isda_calculator_reconcile_single_quote(self):
        print("Testing ISDA engine calculations for a single credit-default swap record (reconciliation)...")
        saved_eval_date = ql.Settings.instance().evaluationDate
        trade_date = ql.Date(26, ql.July, 2021)
        ql.Settings.instance().evaluationDate = trade_date

        isda_rate_helpers = []
        dep_tenors = [1, 3, 6, 12]; dep_quotes = [-0.0056,-0.005440,-0.005190,-0.004930]
        for i in range(len(dep_tenors)):
            isda_rate_helpers.append(ql.DepositRateHelper(dep_quotes[i], ql.Period(dep_tenors[i], ql.Months), 2,
                                     ql.WeekendsOnly(), ql.ModifiedFollowing, False, ql.Actual360()))

        swap_tenors = [2,3,4,5,6,7,8,9,10,12,15,20,30]
        swap_quotes = [-0.004820,-0.004420,-0.003990,-0.003520,-0.002970,-0.002370,-0.001760,
                       -0.001140,-0.000540,0.000570,0.001880,0.002940,0.002820]
        isda_ibor = ql.IborIndex("IsdaIborEUR", ql.Period(6, ql.Months), 2, ql.EURCurrency(),
                                 ql.WeekendsOnly(), ql.ModifiedFollowing, False, ql.Actual360()) # EUR, Annual for swaps
        for i in range(len(swap_tenors)):
            isda_rate_helpers.append(ql.SwapRateHelper(swap_quotes[i], ql.Period(swap_tenors[i], ql.Years),
                                     ql.WeekendsOnly(), ql.Annual, ql.ModifiedFollowing, # Annual fixed leg
                                     ql.Thirty360(ql.Thirty360.BondBasis), isda_ibor))

        discount_curve = ql.RelinkableYieldTermStructureHandle()
        discount_curve.linkTo(ql.PiecewiseYieldCurveLogLinearDiscount(
                0, ql.WeekendsOnly(), isda_rate_helpers, ql.Actual365Fixed()))

        probability_curve = ql.RelinkableDefaultProbabilityTermStructureHandle()
        instrument_maturity = ql.Date(20, ql.June, 2026)
        coupon = 0.01; conventional_spread = 0.006713; recovery = 0.4
        nominal = 1e6; markit_value = -16070.7; expected_accrual = 1000.0; tolerance = 1.0e-3

        quoted_trade = ql.MakeCreditDefaultSwap(instrument_maturity, conventional_spread) \
                         .withNominal(nominal).makeCreditDefaultSwap()

        h = quoted_trade.impliedHazardRate(0.0, discount_curve, ql.Actual365Fixed(),
                                           recovery, 1e-10, ql.CreditDefaultSwap.ISDA)
        probability_curve.linkTo(ql.FlatHazardRate(0, ql.WeekendsOnly(), ql.QuoteHandle(ql.SimpleQuote(h)), ql.Actual365Fixed()))

        engine = ql.IsdaCdsEngine(probability_curve, recovery, discount_curve, None,
                                  ql.IsdaCdsEngine.Taylor, ql.IsdaCdsEngine.HalfDayBias, ql.IsdaCdsEngine.Piecewise)

        conventional_trade = ql.MakeCreditDefaultSwap(instrument_maturity, coupon) \
                               .withNominal(nominal).withPricingEngine(engine).makeCreditDefaultSwap()

        npv = conventional_trade.NPV()
        calculated_upfront_amount = conventional_trade.notional() * conventional_trade.fairUpfront()
        # df calculation in C++ seems to be for context, not direct use in assertions on upfront
        # df = calculated_upfront_amount / npv # if npv is not zero

        # Check NPV against Markit value
        self.assertAlmostEqual(npv, markit_value, delta=abs(markit_value * tolerance), # Relative tolerance
                               msg=f"ISDA Reconcile NPV: expected {markit_value}, got {npv}")

        # Check calculated upfront against Markit value (after discount factor adjustment implicitly)
        # The test implies markitValue is the NPV, and we need to check fairUpfront * notional
        # QL_CHECK_CLOSE(calculated_upfront, df * markitValue, tolerance);
        # This line from C++ is a bit circular if calculated_upfront is derived from npv.
        # Let's check fairUpfront * notional against markitValue, assuming markitValue is the target for PV of upfront.
        # However, the problem implies markitValue is the CDS NPV.
        # The C++ `calculated_upfront = conventionalTrade->notional() * conventionalTrade->fairUpfront();`
        # `df = calculated_upfront / npv`
        # `QL_CHECK_CLOSE(calculated_upfront, df * markitValue, tolerance);` -> `calculated_upfront == (calculated_upfront / npv) * markitValue`
        # -> `1 == markitValue / npv` -> `npv == markitValue`. So this check is redundant if first check passes.

        # Check accrual rebate calculation
        # The C++ derived_accrual is: df * (npv - defaultLegNPV - couponLegNPV)
        # This essentially means (calculated_upfront / npv) * (accrual_rebate_component_of_npv * npv / upfront_pv_factor)
        # It's simpler to get the accrual directly.
        calculated_accrual_amount = conventional_trade.accrualRebate().amount()
        self.assertAlmostEqual(calculated_accrual_amount, expected_accrual, delta=abs(expected_accrual * tolerance),
                               msg=f"ISDA Reconcile Accrual: expected {expected_accrual}, got {calculated_accrual_amount}")

        settlement_date = conventional_trade.accrualRebate().date()
        expected_settlement_date = ql.WeekendsOnly().advance(trade_date, 3, ql.Days)
        self.assertEqual(settlement_date, expected_settlement_date,
                         msg=f"ISDA Reconcile Settlement Date: expected {expected_settlement_date}, got {settlement_date}")

        ql.Settings.instance().evaluationDate = saved_eval_date


    def test_isda_calculator_reconcile_single_with_issue_date_in_the_past(self):
        print("Testing ISDA engine calculations for a single credit-default swap with issue date in the past...")
        saved_eval_date = ql.Settings.instance().evaluationDate
        value_date = ql.Date(26, ql.July, 2021)
        ql.Settings.instance().evaluationDate = value_date

        trade_date_past = ql.Date(20, ql.July, 2019) # This is the 'effective' start date for the CDS

        # Yield curve setup (same as previous ISDA test)
        isda_rate_helpers = []
        dep_tenors = [1, 3, 6, 12]; dep_quotes = [-0.0056,-0.005440,-0.005190,-0.004930]
        for i in range(len(dep_tenors)):
            isda_rate_helpers.append(ql.DepositRateHelper(dep_quotes[i], ql.Period(dep_tenors[i], ql.Months), 2,
                                     ql.WeekendsOnly(), ql.ModifiedFollowing, False, ql.Actual360()))
        swap_tenors = [2,3,4,5,6,7,8,9,10,12,15,20,30]
        swap_quotes = [-0.004820,-0.004420,-0.003990,-0.003520,-0.002970,-0.002370,-0.001760,
                       -0.001140,-0.000540,0.000570,0.001880,0.002940,0.002820]
        isda_ibor = ql.IborIndex("IsdaIborEURPast", ql.Period(6, ql.Months), 2, ql.EURCurrency(),
                                 ql.WeekendsOnly(), ql.ModifiedFollowing, False, ql.Actual360())
        for i in range(len(swap_tenors)):
            isda_rate_helpers.append(ql.SwapRateHelper(swap_quotes[i], ql.Period(swap_tenors[i], ql.Years),
                                     ql.WeekendsOnly(), ql.Annual, ql.ModifiedFollowing,
                                     ql.Thirty360(ql.Thirty360.BondBasis), isda_ibor))
        discount_curve = ql.RelinkableYieldTermStructureHandle()
        discount_curve.linkTo(ql.PiecewiseYieldCurveLogLinearDiscount(
                0, ql.WeekendsOnly(), isda_rate_helpers, ql.Actual365Fixed()))

        probability_curve = ql.RelinkableDefaultProbabilityTermStructureHandle()
        instrument_maturity = ql.Date(20, ql.June, 2026)
        coupon = 0.01; conventional_spread = 0.006713; recovery = 0.4
        nominal = 1e6; markit_value_past_issue = -17070.77; expected_accrual_past_issue = 0.0; tolerance = 1.0e-3

        quoted_trade = ql.MakeCreditDefaultSwap(instrument_maturity, conventional_spread) \
                         .withNominal(nominal).makeCreditDefaultSwap()
        h = quoted_trade.impliedHazardRate(0.0, discount_curve, ql.Actual365Fixed(),
                                           recovery, 1e-10, ql.CreditDefaultSwap.ISDA)
        probability_curve.linkTo(ql.FlatHazardRate(0, ql.WeekendsOnly(), ql.QuoteHandle(ql.SimpleQuote(h)), ql.Actual365Fixed()))

        engine = ql.IsdaCdsEngine(probability_curve, recovery, discount_curve, None,
                                  ql.IsdaCdsEngine.Taylor, ql.IsdaCdsEngine.HalfDayBias, ql.IsdaCdsEngine.Piecewise)

        # Use withTradeDate to set the CDS start date in the past
        conventional_trade_past_issue = ql.MakeCreditDefaultSwap(instrument_maturity, coupon) \
                                           .withNominal(nominal) \
                                           .withPricingEngine(engine) \
                                           .withTradeDate(trade_date_past).makeCreditDefaultSwap() # Key difference

        npv_past_issue = conventional_trade_past_issue.NPV()

        # When trade date is in the past relative to eval date, accrual rebate might not apply or be zero.
        # The C++ test's `calculated_accrual = npv - defaultLegNPV - couponLegNPV;`
        # This is essentially the PV of the upfront payment if the CDS has a non-zero upfront.
        # If the CDS is structured with a running spread only (no upfront), this should be zero.
        # The test name implies that the issue date is in the past, so the accrual for the current period might be handled differently
        # or the "accrual rebate on default" which is part of default leg calculation is what is targeted.
        # Let's check NPV directly and then the accrualRebate component.

        self.assertAlmostEqual(npv_past_issue, markit_value_past_issue, delta=abs(markit_value_past_issue * tolerance),
                               msg=f"ISDA Past Issue NPV: expected {markit_value_past_issue}, got {npv_past_issue}")

        accrual_rebate_obj_past = conventional_trade_past_issue.accrualRebate()
        actual_accrual_amount = accrual_rebate_obj_past.amount() if accrual_rebate_obj_past else 0.0

        self.assertAlmostEqual(actual_accrual_amount, expected_accrual_past_issue, delta=tolerance, # Using absolute tolerance for zero expected
                               msg=f"ISDA Past Issue Accrual Rebate: expected {expected_accrual_past_issue}, got {actual_accrual_amount}")

        ql.Settings.instance().evaluationDate = saved_eval_date


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    # TopLevelFixture in C++ might set a default evaluation date.
    # Python tests usually manage this explicitly or rely on QL's default.
    # Using a saved_eval_date pattern within tests is good practice.
    unittest.main(argv=['first-arg-is-ignored'], exit=False)